In [1]:
import os

In [5]:
%pwd

'c:\\Users\\Krish Goyani\\Desktop\\DS Projects\\AIDRP-AI_Driven_Diabetes_Readmission_Prevention'

In [4]:
os.chdir('../')

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvalutionConfig:
    root_dir:Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    all_params: dict
    target_column : str

In [8]:
from AIDRP.constants import *
from AIDRP.utils.common import read_yaml, create_directories, save_json

class ConfigurationManagaer:
    def __init__(self,
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = config_filepath
        self.params = params_filepath
        self.schema = schema_filepath

        create_directories([self.config.artifacts_root])
        

    def get_model_evalution_config(self)->ModelEvalutionConfig:
        config = self.config.model_evalution
        params = self.params.CatBoostClassifier
        schema = self.schema.TARGET_COLUMN

        model_evalution_config = ModelEvalutionConfig(
            root_dir= config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            schema = schema.readmitted
        )

        return model_evalution_config


In [9]:
from sklearn.metrics import f1_score
import pandas as pd
import joblib

class ModelEvalution:
    def __init__(self,config: ModelEvalutionConfig) :
        self.config = config

    def eval_metrics(self,actual,pred):
        f1 = f1_score(actual,pred,average='micro')
        return f1
    
    def save_result(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.metric_file_name)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        predicted_values = model.predict(X_test)

        f1 = self.eval_metrics(y_test,predicted_values)

        # Saving metrics as local
        score = {"F1 Score" : f1}
        save_json(path=Path(self.config.metric_file_name), data=score)

C:\Users\Krish Goyani\AppData\Local\Temp\ipykernel_8356\3678383159.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
try:
    config = ConfigurationManagaer()
    model_evalution_config = config.get_model_evalution_config()
    model_evalution_config = ModelEvalution(model_evalution_config)
    model_evalution_config.save_result()

except Exception as e:
    raise e